In [12]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [13]:
import sys
import os
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from bpe import BayesPE  # BayesPE class
from llm_model import LLM
import evaluation  # Evaluation functions
import constants

In [14]:
# Define task instructions
instructions = [
    "Classify the sentiment of the following movie review into one of the given categories.",
    "Determine the emotional tone expressed in the movie review excerpt below.",
    "Assign a sentiment label to the text based on its overall attitude.",
    "Analyze the review and select the appropriate sentiment category it falls under.",
    "What is the sentiment conveyed by this portion of the movie review? Choose from the specified classes.",
    "Label the following movie review extract with its correct sentiment: positive, negative, or neutral.",
    "Identify and classify the sentiment expressed in the review passage below.",
    "Based on the language and tone of the review, determine the correct sentiment label.",
    "Select the sentiment category that best matches the opinion expressed in the review snippet."
]

In [15]:
# Load SST2 train and test data
df_train = pd.read_csv('train_sst2.csv')
df_test = pd.read_csv('test_sst2.csv')
n_train = 50000  
n_in_context = 5  

n_total_in_context = len(instructions) * n_in_context  
n_val=100
df_train_actual = df_train.iloc[:n_train] 
df_in_context_base = df_train.iloc[n_train:n_train + n_total_in_context]
df_val = df_train.iloc[n_train + n_total_in_context:n_train+n_total_in_context+n_val]
df_test_actual = df_test.iloc[:]  

gt_labels_train = df_train_actual.iloc[:, 2].values.astype(int) 
samples_train = df_train_actual.iloc[:, 1].values 
gt_labels_val = df_val.iloc[:, 2].values.astype(int) 
samples_val = df_val.iloc[:, 1].values 
gt_labels_test = df_test_actual.iloc[:, 2].values.astype(int)
samples_test = df_test_actual.iloc[:, 1].values 

# **Prepare Unique In-Context Examples Per Instruction**
for i in range(len(instructions)):  
    start_idx = i * n_in_context
    end_idx = (i + 1) * n_in_context
    df_in_context = df_in_context_base.iloc[start_idx:end_idx]

    samples_in_context_i = df_in_context.iloc[:, 1].values
    gt_labels_in_context_i = df_in_context.iloc[:, 2].values.astype(int)

    if i == 0:
        samples_in_context = np.expand_dims(samples_in_context_i, axis=1)
        gt_labels_in_context = np.expand_dims(gt_labels_in_context_i, axis=1)
    else:
        samples_in_context = np.concatenate((samples_in_context, np.expand_dims(samples_in_context_i, axis=1)), axis=1)
        gt_labels_in_context = np.concatenate((gt_labels_in_context, np.expand_dims(gt_labels_in_context_i, axis=1)), axis=1)


In [16]:
# Define a prompt formatting class for sentiment classification and initializes an LLM-based classifier
class PromptFormatting(object):
    def __init__(self):
        self.INSTRUCTION = 'Classify the sentiment of the following movie review into one of the given categories.'
        self.CLASSES = ['negative', 'positive']
        self.CLASSES_FOR_MATCHING = [self.CLASSES, ['neg', 'pos'], ['1', '2']]
        self.CLASSES_TEXT = '''1. {}\n2. {}'''.format(self.CLASSES[0], self.CLASSES[1])

    def format_instruction(self, instruction):
        return '''{}\n{}\n'''.format(instruction, self.CLASSES_TEXT)

    def format_content(self, content):
        return '''review: {}\nthe review is '''.format(content)

prompt_formatting = PromptFormatting()



# Initialize BayesPE (Teacher Model)
bayespe_classifier = BayesPE(
    model_name="mistralai/Mistral-7B-Instruct-v0.3", 
    prompt_formatting=prompt_formatting,
    instructions=instructions, 
    few_shot_texts_sets=samples_in_context, 
    few_shot_labels_sets=gt_labels_in_context, 
    use_reduced_precision=True
)

# Print example prompt
bayespe_classifier.print_prompt_example()

# Optimize prompt weights
bayespe_classifier.optimise_weights(samples_val, gt_labels_val)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

EXAMPLE 1:
Classify the sentiment of the following movie review into one of the given categories.
1. negative
2. positive

review: glow 
the review is positive

EXAMPLE 2:
Classify the sentiment of the following movie review into one of the given categories.
1. negative
2. positive

review: a classical dramatic animated feature 
the review is positive

EXAMPLE 3:
Classify the sentiment of the following movie review into one of the given categories.
1. negative
2. positive

review: best espionage picture 
the review is positive

EXAMPLE 4:
Classify the sentiment of the following movie review into one of the given categories.
1. negative
2. positive

review: drag on for nearly three hours 
the review is negative

EXAMPLE 5:
Classify the sentiment of the following movie review into one of the given categories.
1. negative
2. positive

review: the entire point of a shaggy dog story , of course , is that it goes nowhere , and 
the review is negative

EXAMPLE 6:
Classify the sentiment of the

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.30it/s]


inference for promt 2 out of 9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 14.75it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.79it/s]


inference for promt 9 out of 9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.73it/s]


iteration 0, loss: 10.080135552103378


array([0.11324611, 0.06342231, 0.10655718, 0.06697492, 0.07831271,
       0.09093669, 0.07694805, 0.11141872, 0.29218334], dtype=float32)

In [6]:
# Get prompt weights and prompt wise probabilities on train data
_,probs,weights = bayespe_classifier.forward(samples_train, n_forward_passes=9)

inference for promt 1 out of 9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [30:13<00:00, 27.57it/s]


inference for promt 2 out of 9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [30:38<00:00, 27.19it/s]


inference for promt 3 out of 9


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [30:57<00:00, 26.92it/s]


inference for promt 8 out of 9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [30:31<00:00, 27.31it/s]


inference for promt 9 out of 9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [30:56<00:00, 26.93it/s]


In [ ]:
torch.save(probs,'sst2_probs.pt')
torch.save(weights,'sst2_prompt_weights.pt')

In [18]:
# Evaluate BayesPE performance on sst2 test data
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
teacher_probs,_,_ = bayespe_classifier.forward(samples_test, n_forward_passes=9)
end.record()

# Wait for the events to be recorded
torch.cuda.synchronize()

# Report in seconds
elapsed_time_ms = start.elapsed_time(end)  # in milliseconds
elapsed_time_sec = elapsed_time_ms / 1000  # convert to seconds

print(f"Inference time: {elapsed_time_sec:.4f} seconds")
print(teacher_probs[:10, :])
f1_score = evaluation.compute_metric(gt_labels_test, teacher_probs, metric='f1')
ece = evaluation.compute_metric(gt_labels_test, teacher_probs, metric='ece')
print('Teacher f1-score: {}, Teacher ECE: {}'.format(f1_score, ece))

inference for promt 1 out of 9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 872/872 [01:00<00:00, 14.38it/s]


inference for promt 2 out of 9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 872/872 [01:04<00:00, 13.56it/s]


inference for promt 3 out of 9


...

inference for promt 8 out of 9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 872/872 [01:03<00:00, 13.64it/s]


inference for promt 9 out of 9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 872/872 [01:05<00:00, 13.29it/s]


Inference time: 577.3881 seconds
[[2.15308646e-05 9.99978484e-01]
 [9.96954942e-01 3.04507307e-03]
 [8.22373726e-05 9.99917778e-01]
 [5.46948111e-05 9.99945320e-01]
 [9.99416429e-01 5.83586077e-04]
 [2.76974521e-04 9.99723040e-01]
 [9.99103096e-01 8.96918986e-04]
 [9.87049482e-01 1.29505324e-02]
 [5.05681623e-05 9.99949447e-01]
 [9.99733136e-01 2.66879291e-04]]
Teacher f1-score: 0.9552751705390573, Teacher ECE: 0.029035435989499092
